In [20]:
from nltk.corpus import brown
import nltk

In [9]:
with open('movie_summaries_bottom_250.txt', 'r') as myfile:
    data=myfile.readlines()
#print(summary[0])

In [31]:
#strip any proper names from a text...unfortunately right now this is yanking the first word from a sentence too.
import string
def strip_proppers(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()
#strip any proper nouns (NNP) or plural proper nouns (NNPS) from a text
from nltk.tag import pos_tag


preprocess = [strip_proppers(doc) for doc in data]


In [40]:
#Paramter
NUM_TOPICS = 4
#Convert a collection of text documents to a matrix of token counts
vectorizer = CountVectorizer(
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(preprocess)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Different topic distribution in the first document
print(lda_Z[0])


c:\anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


(249, 4)
[ 0.00251003  0.00254301  0.00254756  0.9923994 ]


In [41]:
def print_topics(model, vectorizer, top_n=4):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('shark', 20.171885615234462), ('story', 10.737477146562107), ('young', 10.69589032758134), ('brother', 9.8571023585884454)]
Topic 1:
[('evil', 11.37253344327222), ('young', 10.123962365269035), ('man', 9.8647388090709693), ('ship', 9.1720725094147646)]
Topic 2:
[('life', 27.538913836673927), ('love', 24.61461402958847), ('young', 23.894281521682451), ('com', 22.518346596168094)]
Topic 3:
[('world', 42.093299504193304), ('evil', 38.160436070604781), ('family', 31.650268805013244), ('save', 21.278636054194124)]


In [42]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups 
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances


In [44]:
svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(data_vectorized)
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], range(len(data))
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
text_font_size="8pt", text_color="#555555",
source=source, text_align='center')
plot = figure(plot_width=300, plot_height=300)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)


In [45]:
svd = TruncatedSVD(n_components=2)
words_2d = svd.fit_transform(data_vectorized.T)
 
df = pd.DataFrame(columns=['x', 'y', 'word'])
df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], vectorizer.get_feature_names()
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [46]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

c:\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
2      33.110260        1       1  343.599976  -51.167015
3      31.597861        1       2  234.853607 -424.282471
0      18.550850        1       3  -29.513216   57.586979
1      16.741029        1       4 -138.259491 -315.528381, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
3547  Default  15.000000      shark  15.000000  30.0000  30.0000
1348  Default  41.000000       evil  41.000000  29.0000  29.0000
1429  Default  30.000000     family  30.000000  28.0000  28.0000
4460  Default  46.000000      world  46.000000  27.0000  27.0000
2309  Default  29.000000       love  29.000000  26.0000  26.0000
3557  Default  11.000000       ship  11.000000  25.0000  25.0000
4280  Default   7.000000    vampire   7.000000  24.0000  24.0000
3114  Default   8.000000       race   8.000000  23.0000  23.0000
3420  Default  24.000000       save  24.000000  22.0000  22.0000
1702  Default  12.000000       gold  12.000000  21.0000  21.0000
1047  Default  12.000000     desert  12.000000  20.0000  20.0000
151   Default   5.000000  apartment   5.000000  19.0000  19.0000
694   Default  11.000000       city  11.000000  18.0000  18.0000
4130  Default  18.000000   treasure  18.000000  17.0000  17.0000
421   Default   6.000000      blood   6.000000  16.0000  16.0000
238   Default   9.000000     attack   9.000000  15.0000  15.0000
4094  Default  22.000000       town  22.000000  14.0000  14.0000
2960  Default  10.000000   powerful  10.000000  13.0000  13.0000
4446  Default  20.000000      woman  20.000000  12.0000  12.0000
4496  Default   8.000000    zombies   8.000000  11.0000  11.0000
3601  Default   9.000000     sister   9.000000  10.0000  10.0000
2162  Default  11.000000     killed  11.000000   9.0000   9.0000
2304  Default  18.000000       lost  18.000000   8.0000   8.0000
2945  Default   6.000000       porn   6.000000   7.0000   7.0000
2427  Default   8.000000    members   8.000000   6.0000   6.0000
1925  Default  15.000000    hotmail  15.000000   5.0000   5.0000
434   Default   5.000000       boat   5.000000   4.0000   4.0000
563   Default   6.000000       camp   6.000000   3.0000   3.0000
1938  Default   5.000000   humanity   5.000000   2.0000   2.0000
507   Default  14.000000    brother  14.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
4292   Topic4   3.183201  vengeance   5.315324   1.2746  -6.4250
2229   Topic4   3.001892        led   5.117886   1.2538  -6.4836
98     Topic4   3.649825      alien   6.512499   1.2083  -6.2882
2960   Topic4   5.224190   powerful  10.088563   1.1292  -5.9296
3557   Topic4   5.792789       ship  11.822071   1.0740  -5.8262
2514   Topic4   2.708767       moon   4.810630   1.2130  -6.5864
2547   Topic4   2.096381   murdered   3.425516   1.2963  -6.8426
100    Topic4   3.884624      alive   7.845140   1.0844  -6.2258
2140   Topic4   4.121916      joins   8.547206   1.0580  -6.1665
385    Topic4   4.555976        big  10.241611   0.9773  -6.0664
694    Topic4   4.735031       city  11.184368   0.9278  -6.0279
3705   Topic4   3.605663      space   8.001186   0.9902  -6.3003
910    Topic4   2.889469       crew   5.634758   1.1194  -6.5218
4483   Topic4   5.107596      years  17.751479   0.5416  -5.9521
1003   Topic4   4.597024    decides  15.078737   0.5994  -6.0574
1348   Topic4   7.182530       evil  41.839312   0.0251  -5.6112
2359   Topic4   6.230255        man  33.496085   0.1053  -5.7534
4481   Topic4   3.524033       year   9.241179   0.8232  -6.3232
4488   Topic4   6.393972      young  42.280113  -0.1017  -5.7275
1615   Topic4   5.600083    friends  32.364438   0.0330  -5.8601
376    Topic4   4.392784       best  17.519904   0.4039  -6.1029
1454   Topic4   5.013321     father  27.717743   0.0773  -5.9708
4062   Topic4   5.375101       tim